### 데이터 분석 기본 - 텍스트 생성 모델 이해
  ▶ 생성 AI 기본 이해 <br>
  ▶ 생성 AI 기본 실습 - 텍스트 생성 및 요약 실습

[학습 목표]
 - Transformers 딥러닝 모델 이해 및 라이브러리 설치
 - 텍스트 생성 및 요약을 실습해 봅니다.

### Transformers 딥러닝 모델
 - Transformers는 자연어 처리(NLP) 및 다양한 시퀀스 데이터 태스크에 사용되는 딥러닝 모델 아키텍처입니다. 2017년 구글 브레인 팀의 "Attention Is All You Need" 논문에서 소개되었습니다.
 - [주요 특징]
   - 셀프 어텐션(Self-Attention) 메커니즘
   - 병렬 연산 구조
     - 순환 신경망(RNN)과 달리 입력 전체를 한번에 처리할 수 있어 연산 속도가 빠르다.
   - 다양한 태스크 적용 가능
     - 번역, 요약, 질의응답 등 다양한 NLP 태스크에 적용 가능.
 - 대표적인 Transformer 기반 모델
   - BERT, GPT, T5, BART 등
   - GPT-3의 성공 이후 Transformer아키텍처가 널리 사용되고 있음.

In [1]:
# 다양한 Transformer 모델들이 구현되어 있어 쉽게 활용 가능.
# Hugging Face의 Transformers 라이브러리를 설치하기 위한 명령어
!pip install transformers

### GPT-2 모델 로드

In [2]:
# 라이브러리 불러오기
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [3]:
# 모델과 토크나이저 초기화
# GPT2Tokenizer는 GPT-2 모델에 사용되는 토크나이저 클래스
# from_pretrained('gpt2')는 사전 훈련된 GPT-2 토크나이저 가중치를 다운로드하여 초기화
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# GPT2LMHeadModel은 GPT-2 언어 모델의 구조를 정의하는 클래스
# from_pretrained('gpt2')는 사전 훈련된 GPT-2 모델 가중치를 다운로드하여 초기화
model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# GPU 사용 설정 (가능한 경우)
# PyTorch 모델을 GPU 또는 CPU에 할당하기 위한 작업
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# device에 모델을 할당
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### 텍스트 생성 함수 정의

In [5]:
### 텍스트 생성 함수 정의
def generate_text(prompt, length=50):
    # 입력 프롬프트 전처리
    # 프롬프트를 토큰화하고 PyTorch 텐서로 변환
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # 인코딩된 입력을 앞서 지정한 device(GPU 또는 CPU)로 이동
    inputs = inputs.to(device)

    # 모델의 generate 메서드를 호출하여 입력에 대한 텍스트를 생성
    # num_return_sequences : 몇 개의 다른 결과물을 출력할지 제어
    outputs = model.generate(inputs, max_length=length, num_return_sequences=1)

    # 생성된 텍스트를 디코딩 - generate로 생성된 것은 숫자열 이를 실제 사람이 이해할 수 있는 실제 텍스트로 만든다.
    # skip_special_tokens=True -> 특수 토큰을 제외한다.
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

In [6]:
# 텍스트 생성 예시
prompt = "The future of artificial intelligence is"
print(generate_text(prompt, length=100))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The future of artificial intelligence is uncertain.

"We're not sure what the future will look like," said Dr. Michael S. Schoenfeld, a professor of computer science at the University of California, Berkeley. "But we're not sure what the future will look like for artificial intelligence."

The future of artificial intelligence is uncertain.

"We're not sure what the future will look like," said Dr. Michael S. Schoenfeld, a professor of computer science


In [7]:
# 텍스트 생성 예시
prompt = "너는 누구니?"
print(generate_text(prompt, length=100))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


너는 누구니?

아는 누구니? 아는 누구니? 아는 누구니? 아는 누구니? 아는 누구니? 아는 �


In [8]:
prompt = "Please explain about Deep Learning"
print(generate_text(prompt, length=100))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Please explain about Deep Learning.

Deep Learning is a new field of research that has been around for a long time. It is a new field of research that has been around for a long time. It is a new field of research that has been around for a long time. It is a new field of research that has been around for a long time. It is a new field of research that has been around for a long time. It is a new field of research that has been around


```
딥 러닝(Deep Learning)은 오랫동안 지속되어 온 새로운 연구 분야입니다. 그것은 오랫동안 지속되어 온 새로운 연구 분야입니다. 그것은 새로운 연구 분야입니다
```

### (2) 텍스트 요약 실습


### 요약

In [10]:
from transformers import pipeline, set_seed

In [11]:
set_seed(42) # 재현성을 위한 시드 설정

# Hugging Face의 pipeline 기능을 사용하여 사전 훈련된 BART 모델을 로드
# 'facebook/bart-large-cnn'은 Facebook AI에서 공개한 BART 모델로, 텍스트 요약 태스크에 특화
# 이 모델은 텍스트 요약 태스크에 특화
summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

text = """
Deep learning is a subfield of artificial intelligence that mimics the way the human brain works to enable computers to learn from data.
It uses multiple layers of neural networks to discover complex patterns and relationships in data, and is utilized for tasks such as image recognition, speech recognition, and language translation.
Deep learning models require large amounts of data and substantial computing power, allowing them to learn independently and improve accuracy over time.
This technology has rapidly advanced in recent years and has become a core component of many modern technologies.
"""

# 원문 텍스트를 입력으로 받아 BART 모델이 최대 130 토큰, 최소 30 토큰 길이의 요약문을 생성
summary_text = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
print(summary_text)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Your max_length is set to 130, but your input_length is only 110. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)


Deep learning is a subfield of artificial intelligence. It mimics the way the human brain works to enable computers to learn from data. It uses multiple layers of neural networks to discover complex patterns and relationships in data.


### 결과
```
딥 러닝은 인간의 뇌가 작동하는 방식을 모방한 인공 지능의 하위 분야입니다.
여러 층의 신경망을 사용하여 데이터의 복잡한 패턴과 관계를 발견합니다. 딥 러닝 모델은 많은 양의 데이터와 상당한 컴퓨팅 능력을 필요로 합니다.
```

In [12]:
text = """
딥 러닝은 인간의 뇌가 작동하는 방식을 모방하여 컴퓨터가 데이터로부터 학습할 수 있도록 하는 인공 지능의 하위 분야입니다.
여러 층의 신경망을 사용하여 데이터의 복잡한 패턴과 관계를 발견하고, 이미지 인식, 음성 인식, 언어 번역 등의 작업에 활용됩니다.
딥 러닝 모델은 많은 양의 데이터와 상당한 컴퓨팅 능력이 필요하므로 독립적으로 학습하고 시간이 지남에 따라 정확도를 향상시킬 수 있습니다.
이 기술은 최근 몇 년 동안 빠르게 발전하여 많은 현대 기술의 핵심 구성 요소가 되었습니다.
"""

# 원문 텍스트를 입력으로 받아 BART 모델이 최대 130 토큰, 최소 30 토큰 길이의 요약문을 생성
summary_text = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
print(summary_text)


ታ  ት  ‘ 학’  하   ‘‘” �’ '”’  ”   “”      ‘  ’,  ””, ”,     , “,  ;. ”. “;.   .  ,.  .,   “, . �. , ”.  
